# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [41]:
from azureml.core import Workspace, Experiment, Environment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [43]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'student-math-experiment-automl'

experiment=Experiment(ws, experiment_name)

# Setting up the enviornment
env = Environment.from_existing_conda_environment(name="my-captured-notebook-env", conda_environment_name="azureml_py38")
env.register(workspace=ws)

Exporting conda specifications for existing conda environment: azureml_py38


{
    "assetId": "azureml://locations/norwayeast/workspaces/7dcbcfea-228e-4d54-94e2-af79d5ae3fd0/environments/my-captured-notebook-env/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20240709.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "my-captur

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [3]:
from azureml.core import Dataset

# Load dataset from Azure ML workspace
df = Dataset.get_by_name(workspace=ws, name="student-math")


In [4]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 15,  
    "primary_metric": "normalized_mean_absolute_error",  
    "n_cross_validations": 5, 
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task="regression",  
    training_data=df,  
    label_column_name="G3", 
    compute_target="notebook-compute", 
    enable_early_stopping=True,
    featurization="auto",  
    **automl_settings
) 

In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
remote_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
student-math-experiment,AutoML_e7050916-dc17-4463-9557-6052ab249c0f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
student-math-experiment,AutoML_e7050916-dc17-4463-9557-6052ab249c0f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
from azureml.widgets import RunDetails

# Show run details in the notebook
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [34]:
best_run, best_model = remote_run.get_output()

In [38]:
import joblib

#TODO: Save the best model
print("Best Run ID:", best_run.id)
print("Best Model Properties:")
for key, value in best_run.get_metrics().items():
    print(f"{key}: {value}")


model_filename = "best_automl_model.pkl"
joblib.dump(best_model, model_filename)

Best Run ID: AutoML_e7050916-dc17-4463-9557-6052ab249c0f_30
Best Model Properties:
mean_absolute_error: 1.2302216132242925
mean_absolute_percentage_error: 9.875926092171118
median_absolute_error: 0.7791784933329347
r2_score: 0.8188206318517042
normalized_median_absolute_error: 0.03895892466664673
normalized_root_mean_squared_log_error: 0.15419519496424933
normalized_root_mean_squared_error: 0.09456412009003883
root_mean_squared_error: 1.8912824018007766
spearman_correlation: 0.940327142994412
normalized_mean_absolute_error: 0.061511080661214614
root_mean_squared_log_error: 0.4694507308577949
explained_variance: 0.8197721332387727
residuals: aml://artifactId/ExperimentRun/dcid.AutoML_e7050916-dc17-4463-9557-6052ab249c0f_30/residuals
predicted_true: aml://artifactId/ExperimentRun/dcid.AutoML_e7050916-dc17-4463-9557-6052ab249c0f_30/predicted_true


['best_automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [39]:
from azureml.core.model import Model
import os

# Ensure the model file exists
model_file = "best_automl_model.pkl"
if not os.path.isfile(model_file):
    raise FileNotFoundError(f"The model file {model_file} does not exist.")

# Register the model
model = Model.register(
    workspace=ws,
    model_name="student-performance-model",
    model_path=model_file,
    description="AutoML trained model for predicting student performance",
    tags={
        "type": "AutoML",
        "task": "regression", 
        "framework": "scikit-learn"
    }
)

print(f"Model registered: Name={model.name}, ID={model.id}, Version={model.version}")


Registering model student-performance-model
Model registered: Name=student-performance-model, ID=student-performance-model:7, Version=7


In [40]:
import sys
import pkg_resources
import importlib

# Gather all imported package names
imported_packages = {pkg.split('.')[0] for pkg in sys.modules.keys() if not pkg.startswith('_')}

# Find installed packages and their versions
installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}

# Print out the packages with versions
for pkg in sorted(imported_packages):
    version = installed_packages.get(pkg.lower())
    if version:
        print(f"{pkg}=={version}")
    else:
        print(f"{pkg} (built-in or unknown version)")


97d2165b493fecec79c65b5c6254dffd4e375528 (built-in or unknown version)
Cython==3.0.10
IPython==8.26.0
OpenSSL (built-in or unknown version)
PIL (built-in or unknown version)
abc (built-in or unknown version)
absl (built-in or unknown version)
adal==1.2.7
aiohappyeyeballs==2.3.5
aiohttp==3.10.1
aiosignal==1.3.1
annotated_types (built-in or unknown version)
anyio==4.4.0
applicationinsights==0.11.10
arch==5.6.0
argcomplete==3.3.0
argparse (built-in or unknown version)
array (built-in or unknown version)
ast (built-in or unknown version)
asttokens==2.4.1
astunparse==1.6.3
async_timeout (built-in or unknown version)
asyncio (built-in or unknown version)
atexit (built-in or unknown version)
attr (built-in or unknown version)
azure (built-in or unknown version)
azureml (built-in or unknown version)
backports (built-in or unknown version)
base64 (built-in or unknown version)
bcrypt==4.2.0
bdb (built-in or unknown version)
binascii (built-in or unknown version)
bisect (built-in or unknown versi

In [44]:
from azureml.core.environment import Environment

env_name = "my-captured-notebook-env" 

try:
    env = Environment.get(workspace=ws, name=env_name)
    print(f"Environment '{env_name}' successfully retrieved.")
except Exception as e:
    print(f"Error retrieving environment '{env_name}': {e}")



Environment 'my-captured-notebook-env' successfully retrieved.


In [47]:
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice

service_name = 'student-performance-predictor'

inference_config = InferenceConfig(entry_script='score.py', environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploying Serice
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],  
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)

print(f"Service state: {service.state}")
print(f"Scoring URI: {service.scoring_uri}")



WebserviceException: WebserviceException:
	Message: Conflict of operation, another operation on same entity is already running in workspace student-performance-ws.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Conflict of operation, another operation on same entity is already running in workspace student-performance-ws."
    }
}

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
input_data = {
    "data": [
        {
            "school": "GP", "sex": "F", "age": 16, "address": "R",
            "famsize": "LE3", "Pstatus": "T", "Medu": 2, "Fedu": 3,
            "Mjob": "other", "Fjob": "teacher", "reason": "home",
            "guardian": "father", "traveltime": 3, "studytime": 3,
            "failures": 2, "schoolsup": "yes", "famsup": "yes",
            "paid": "yes", "activities": "no", "nursery": "yes",
            "higher": "no", "internet": "yes", "romantic": "no",
            "famrel": 1, "freetime": 2, "goout": 4, "Dalc": 5,
            "Walc": 5, "health": 1, "absences": 8, "G1": 15, "G2": 20
        },
        {
            "school": "GP", "sex": "M", "age": 16, "address": "U",
            "famsize": "GT3", "Pstatus": "A", "Medu": 2, "Fedu": 0,
            "Mjob": "health", "Fjob": "health", "reason": "home",
            "guardian": "other", "traveltime": 3, "studytime": 2,
            "failures": 1, "schoolsup": "no", "famsup": "yes",
            "paid": "yes", "activities": "yes", "nursery": "no",
            "higher": "no", "internet": "yes", "romantic": "no",
            "famrel": 1, "freetime": 4, "goout": 4, "Dalc": 1,
            "Walc": 2, "health": 1, "absences": 18, "G1": 8, "G2": 10
        },
        {
            "school": "GP", "sex": "M", "age": 20, "address": "U",
            "famsize": "LE3", "Pstatus": "T", "Medu": 2, "Fedu": 0,
            "Mjob": "teacher", "Fjob": "other", "reason": "reputation",
            "guardian": "other", "traveltime": 3, "studytime": 1,
            "failures": 0, "schoolsup": "no", "famsup": "yes",
            "paid": "yes", "activities": "no", "nursery": "yes",
            "higher": "yes", "internet": "no", "romantic": "yes",
            "famrel": 1, "freetime": 2, "goout": 2, "Dalc": 3,
            "Walc": 2, "health": 1, "absences": 11, "G1": 17, "G2": 13
        }
    ]
}

In [ ]:
import requests
import json

# Replace with your actual endpoint URL
endpoint_url = "https://your-endpoint-url.azurewebsites.net/score"

# If your service requires authentication, include the key
api_key = "your-api-key-if-required"  # Replace with actual key if needed
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}" if api_key else None
}

# Send request to the web service
response = requests.post(endpoint_url, data=json.dumps(input_data), headers=headers)

# Print response
print("Prediction Response:", response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
from azureml.core.webservice import Webservice

# Print logs
print("Service logs:")
print(service.get_logs())

# Delete the service
service.delete()
print(f"Service '{service_name}' has been deleted.")

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
